# Extraction Example
Author: Michael Larson
Last Update: 24 April 2023

This repository is intended to serve as an example for anyone who needs to build an effective area curve. This script is a minimal working example used to extract information from i3 files generated by neutrino-generator or GENIE.

In [25]:
import os, sys, glob
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

try: from tqdm import tqdm
except: tqdm = lambda x: x

import icecube
from icecube import dataclasses, dataio, icetray, simclasses

## Find some files you'd like to generate effective areas from
This can be any set of NuGen or GENIE files. 

In [9]:
oscnext_dir = "/data/ana/LE/oscNext/pass2/genie/level7_flercnn/"
run = "140000"
output_filename = f"oscnext_{run}.npy"

# Let's just grab the first 100 files.
oscnext_files = sorted(glob.glob(os.path.join(oscnext_dir, run, "*")))[:100]

## Grab the OneWeight values
Here I'll define a function to extract the "oneweight" value with it's appropriate scale factors. Note that the 0.7 and 0.3 are specific to older GENIE MC. GENIE files that are produced via genie-reader do NOT need these factors (ie, they're 1.0 for both). NuGen files that don't have `OneWeightPerType` use factors of 0.5 for both.

In [21]:
def get_oneweight(frame, nfiles):
    mcwd = frame['I3MCWeightDict']
    
    # These are newer NuGen files if this is found. It directly includes
    # the fraction of nu and nubar in the OneWeightPerType value, so we 
    # don't need to guess what the right values are.
    if "OneWeightPerType" in mcwd:
        return mcwd['OneWeightPerType'] / mcwd['NEvents'] / nfiles
        
    # GENIE-reader files only include either nu or nubar. No guessing here.
    # Note that the "nevents" comes from a different place, though.
    if 'I3GenieInfo' in frame:
        return mcwd['OneWeight'] / frame['I3GenieInfo'].n_flux_events / nfiles
    
    # No luck. We'll have to guess at the nu/nubar factors, but that shouldn't be
    # too difficult. They've been constant for years. For GENIE-icetray, that's
    # 70/30
    elif any('genie' in key.lower() for key in frame.keys()):
        nu = dataclasses.get_most_energetic_neutrino(frame['I3MCTree'])
        if nu.pdg_encoding > 0: nunubar = 0.7  # Generated with 70% nu
        else:                   nunubar = 0.3  # and 30% nubar
        return mcwd['OneWeight'] / mcwd['NEvents'] / nunubar / nfiles

    # And with older NuGen, this is 50/50
    else:
        nunubar = 0.5
        return mcwd['OneWeight'] / mcwd['NEvents'] / nunubar / nfiles

## And now we do a general loop over i3 files
This is just an example. You could do this in a more efficient way by looking at hdf5 files instead, but here I just show a simple loop over frames.

I'll be storing the results in a numpy structured array since it's easy to save. You could just as easily use pandas or hdf if you choose.

In [22]:
def read_files(files, nfiles=None):
    ow, energy, zen, dec = [], [], [], []
    
    if nfiles is None: nfiles = len(files)

    # Loop over the files
    for f in tqdm(files):
        i3file = dataio.I3File(f)

        # And loop through frames for this file
        while i3file.more():
            frame = i3file.pop_frame()
            if frame.Stop != icetray.I3Frame.Physics: continue

            # Get the relevant information from this frame
            nu = dataclasses.get_most_energetic_neutrino(frame['I3MCTree'])
            ow.append(get_oneweight(frame, nfiles))
            energy.append(nu.energy)
            zen.append(nu.dir.zenith)
            dec.append(nu.dir.zenith - np.pi/2) # Calculate zenith to declination

    # Convert from basic python lists to a numpy structured array
    return np.array(list(zip(energy, zen, dec, ow)), 
                    dtype = [('trueE', float), 
                             ('trueZen', float), 
                             ('trueDec', float), 
                             ('ow', np.float64)])

In [ ]:
oscnext_recarr = read_files(oscnext_files)

## And save the results.

In [7]:
print(f"Number of events found: {oscnext_recarr.shape}")
np.save(output_filename, oscnext_recarr)

Number of events found: (304651,)


## We can do this for upgrade files using the same code.

In [ ]:
# Trigger-level Upgrade NuMu files
upgrade_dir = "/data/sim/IceCubeUpgrade/genie/step4/"
run = "140028"
output_filename = f"upgrade_trigger_{run}.npy"
files = sorted(glob.glob(os.path.join(upgrade_dir, run, "upgrade*")))[:50]
dataset = read_files(files)
np.save(output_filename, dataset)

In [15]:
# Trigger-level Upgrade NuMuBar files
upgrade_dir = "/data/sim/IceCubeUpgrade/genie/step4/"
run = "140128"
output_filename = f"upgrade_trigger_{run}.npy"
files = sorted(glob.glob(os.path.join(upgrade_dir, run, "upgrade*")))[:50]
dataset = read_files(files)
np.save(output_filename, dataset)

  0%|          | 0/50 [00:00<?, ?it/s]

## Or for final-level upgrade analysis files

In [16]:
# QUESO Upgrade NuMu files
upgrade_dir = "/data/sim/IceCubeUpgrade/genie/level4_queso/"
run = "140028"
files = sorted(glob.glob(os.path.join(upgrade_dir, run, "*")))[:50]
output_filename = f"queso_{run}.npy"
dataset = read_files(files)
np.save(output_filename, dataset)

  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
# QUESO Upgrade NuMuBar files
upgrade_dir = "/data/sim/IceCubeUpgrade/genie/level4_queso/"
run = "140128"
files = sorted(glob.glob(os.path.join(upgrade_dir, run, "*")))[:50]
output_filename = f"queso_{run}.npy"
dataset = read_files(files)
np.save(output_filename, dataset)

  0%|          | 0/50 [00:00<?, ?it/s]